# 🎨 Data Designer Tutorial: Seeding Synthetic Data Generation with an External Dataset

#### 📚 What you'll learn

In this notebook, we will demonstrate how to seed synthetic data generation in Data Designer with an external dataset.

If this is your first time using Data Designer, we recommend starting with the [first notebook](/notebooks/1-the-basics/) in this tutorial series.


### 📦 Import the essentials

- The `essentials` module provides quick access to the most commonly used objects.


In [1]:
from data_designer.essentials import (
    DataDesigner,
    DataDesignerConfigBuilder,
    InferenceParameters,
    ModelConfig,
    SeedConfig,
)

### ⚙️ Initialize the Data Designer interface

- `DataDesigner` is the main object is responsible for managing the data generation process.

- When initialized without arguments, the [default model providers](https://nvidia-nemo.github.io/DataDesigner/concepts/models/default-model-settings/) are used.


In [2]:
data_designer_client = DataDesigner()

### 🎛️ Define model configurations

- Each `ModelConfig` defines a model that can be used during the generation process.

- The "model alias" is used to reference the model in the Data Designer config (as we will see below).

- The "model provider" is the external service that hosts the model (see the [model config](https://nvidia-nemo.github.io/DataDesigner/concepts/models/default-model-settings/) docs for more details).

- By default, we use [build.nvidia.com](https://build.nvidia.com/models) as the model provider.


In [3]:
# This name is set in the model provider configuration.
MODEL_PROVIDER = "nvidia"

# The model ID is from build.nvidia.com.
MODEL_ID = "nvidia/nvidia-nemotron-nano-9b-v2"

# We choose this alias to be descriptive for our use case.
MODEL_ALIAS = "nemotron-nano-v2"

# This sets reasoning to False for the nemotron-nano-v2 model.
SYSTEM_PROMPT = "/no_think"

model_configs = [
    ModelConfig(
        alias=MODEL_ALIAS,
        model=MODEL_ID,
        provider=MODEL_PROVIDER,
        inference_parameters=InferenceParameters(
            temperature=0.5,
            top_p=1.0,
            max_tokens=1024,
        ),
    )
]

### 🏗️ Initialize the Data Designer Config Builder

- The Data Designer config defines the dataset schema and generation process.

- The config builder provides an intuitive interface for building this configuration.

- The list of model configs is provided to the builder at initialization.


In [4]:
config_builder = DataDesignerConfigBuilder(model_configs=model_configs)

## 🏥 Prepare a seed dataset

- For this notebook, we'll create a synthetic dataset of patient notes.

- We will _seed_ the generation process with a [symptom-to-diagnosis dataset](https://huggingface.co/datasets/gretelai/symptom_to_diagnosis).

- We already have the dataset downloaded in the [data](../data) directory of this repository.

<br>

> 🌱 **Why use a seed dataset?**
>
> - Seed datasets let you steer the generation process by providing context that is specific to your use case.
>
> - Seed datasets are also an excellent way to inject real-world diversity into your synthetic data.
>
> - During generation, prompt templates can reference any of the seed dataset fields.


In [5]:
# Download sample dataset from Github
import urllib.request

url = "https://raw.githubusercontent.com/NVIDIA/GenerativeAIExamples/refs/heads/main/nemo/NeMo-Data-Designer/data/gretelai_symptom_to_diagnosis.csv"
local_filename, headers = urllib.request.urlretrieve(url, "gretelai_symptom_to_diagnosis.csv")

seed_dataset = SeedConfig(dataset=local_filename)

# Pass the reference to the config builder for use during generation.
config_builder.with_seed_dataset(seed_dataset)

DataDesignerConfigBuilder(
    seed_dataset: 'gretelai_symptom_to_diagnosis.csv'
    seed_dataset_columns: ['diagnosis', 'patient_summary']
)

## 🎨 Designing our synthetic patient notes dataset

- Here we use `add_column` with keyword arguments (rather than imported config objects).

- Generally, we recommend using concrete objects, but this is a convenient shorthand.

- **Note**: The prompt template can reference fields from our seed dataset:
  - `{{ diagnosis }}` - the medical diagnosis from the seed data
  - `{{ patient_summary }}` - the symptom description from the seed data


In [6]:
config_builder.add_column(
    name="patient_sampler",
    column_type="sampler",
    sampler_type="person_from_faker",
)

config_builder.add_column(
    name="doctor_sampler",
    column_type="sampler",
    sampler_type="person_from_faker",
)

config_builder.add_column(
    name="patient_id",
    column_type="sampler",
    sampler_type="uuid",
    params={
        "prefix": "PT-",
        "short_form": True,
        "uppercase": True,
    },
)

config_builder.add_column(
    name="first_name",
    column_type="expression",
    expr="{{ patient_sampler.first_name}}",
)

config_builder.add_column(
    name="last_name",
    column_type="expression",
    expr="{{ patient_sampler.last_name }}",
)


config_builder.add_column(
    name="dob",
    column_type="expression",
    expr="{{ patient_sampler.birth_date }}",
)

config_builder.add_column(
    name="symptom_onset_date",
    column_type="sampler",
    sampler_type="datetime",
    params={"start": "2024-01-01", "end": "2024-12-31"},
)

config_builder.add_column(
    name="date_of_visit",
    column_type="sampler",
    sampler_type="timedelta",
    params={"dt_min": 1, "dt_max": 30, "reference_column_name": "symptom_onset_date"},
)

config_builder.add_column(
    name="physician",
    column_type="expression",
    expr="Dr. {{ doctor_sampler.last_name }}",
)

config_builder.add_column(
    name="physician_notes",
    column_type="llm-text",
    prompt="""\
You are a primary-care physician who just had an appointment with {{ first_name }} {{ last_name }},
who has been struggling with symptoms from {{ diagnosis }} since {{ symptom_onset_date }}.
The date of today's visit is {{ date_of_visit }}.

{{ patient_summary }}

Write careful notes about your visit with {{ first_name }},
as Dr. {{ doctor_sampler.first_name }} {{ doctor_sampler.last_name }}.

Format the notes as a busy doctor might.
""",
    model_alias=MODEL_ALIAS,
    system_prompt=SYSTEM_PROMPT,
)

config_builder.validate()

[18:18:05] [INFO] ✅ Validation passed


DataDesignerConfigBuilder(
    seed_dataset: 'gretelai_symptom_to_diagnosis.csv'
    seed_dataset_columns: ['diagnosis', 'patient_summary']
    sampler_columns: [
        "patient_sampler",
        "doctor_sampler",
        "patient_id",
        "symptom_onset_date",
        "date_of_visit"
    ]
    llm_text_columns: ['physician_notes']
    expression_columns: [
        "first_name",
        "last_name",
        "dob",
        "physician"
    ]
)

### 🔁 Iteration is key – preview the dataset!

1. Use the `preview` method to generate a sample of records quickly.

2. Inspect the results for quality and format issues.

3. Adjust column configurations, prompts, or parameters as needed.

4. Re-run the preview until satisfied.


In [7]:
preview = data_designer_client.preview(config_builder, num_records=2)

[18:18:05] [INFO] 👁️ Preview generation in progress


[18:18:05] [INFO] ✅ Validation passed


[18:18:05] [INFO] ⛓️ Sorting column configs into a Directed Acyclic Graph


[18:18:05] [INFO] 🩺 Running health checks for models...


[18:18:05] [INFO]   |-- 👀 Checking 'nvidia/nvidia-nemotron-nano-9b-v2' in provider named 'nvidia' for model alias 'nemotron-nano-v2'...


...

[18:18:19] [INFO]   |-- 🎲 column: 'symptom_onset_date'


[18:18:19] [INFO]   |-- 🎲 column: 'date_of_visit'


[18:18:19] [INFO]   |-- 🧩 column: 'physician'


[18:18:19] [INFO]   |-- 📝 column: 'physician_notes'


[18:18:19] [INFO] 👏 Preview complete!


In [8]:
# Run this cell multiple times to cycle through the 2 preview records.
preview.display_sample_record()

                                                   Seed Columns                                                    
┏━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Name            ┃ Value                                                                                         ┃
┡━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ diagnosis       │ cervical spondylosis                                                                          │
├─────────────────┼───────────────────────────────────────────────────────────────────────────────────────────────┤
│ patient_summary │ I've been having a lot of pain in my neck and back. I've also been having trouble with my     │
│                 │ balance and coordination. I've been coughing a lot and my limbs feel weak.                    │
└─────────────────┴───────────────────────────────────────────────────────────────────────────────────────────────┘
                                                                                                                   
                                                                                                                   
                                                 Generated Columns                                                 
┏━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Name               ┃ Value                                                                                      ┃
┡━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ patient_sampler    │ {                                                                                          │
│                    │     'uuid': '75229ddd-8f70-4428-8f14-8166fe281f43',                                        │
│                    │     'locale': 'en_US',                                                                     │
│                    │     'first_name': 'Jonathan',                                                              │
│                    │     'last_name': 'Young',                                                                  │
│                    │     'middle_name': None,                                                                   │
│                    │     'sex': 'Male',                                                                         │
│                    │     'street_number': '2492',                                                               │
│                    │     'street_name': 'Scott Gardens',                                                        │
│                    │     'city': 'South Joshuaville',                                                           │
│                    │     'state': 'Missouri',                                                                   │
│                    │     'postcode': '03833',                                                                   │
│                    │     'age': 33,                                                                             │
│                    │     'birth_date': '1992-05-15',                                                            │
│                    │     'country': 'Antarctica (the territory South of 60 deg S)',                             │
│                    │     'marital_status': 'separated',                                                         │
│                    │     'education_level': 'some_college',                                                     │
│                    │     'unit': '',                                                                            │
│                    │     'occupation': 'International aid/development worker',                                  │
│                    │     'phone_number': '952.681.6505

In [9]:
# The preview dataset is available as a pandas DataFrame.
preview.dataset

,diagnosis,patient_summary,patient_sampler,doctor_sampler,patient_id,symptom_onset_date,date_of_visit,first_name,last_name,dob,physician,physician_notes
0,cervical spondylosis,I've been having a lot of pain in my neck and ...,{'uuid': '75229ddd-8f70-4428-8f14-8166fe281f43...,{'uuid': '5dd25b12-9dc8-4e49-bb78-e7764c789b6c...,PT-65199462,2024-02-23,2024-03-12,Jonathan,Young,1992-05-15,Dr. Reed,**Patient Name:** Jonathan Young \n**Date of ...
1,impetigo,I have a rash on my face that is getting worse...,{'uuid': '3d0491f1-6347-410d-a68e-e2ebc3d083af...,{'uuid': '364b95f5-128b-42a5-9ca4-99a34d66c41d...,PT-99EACADA,2024-05-16,2024-05-28,Douglas,Taylor,1980-12-06,Dr. Wise,**Patient Name:** Douglas Taylor \n**Date of ...


### 📊 Analyze the generated data

- Data Designer automatically generates a basic statistical analysis of the generated data.

- This analysis is available via the `analysis` property of generation result objects.


In [10]:
# Print the analysis as a table.
preview.analysis.to_report()

──────────────────────────────────────── 🎨 Data Designer Dataset Profile ─────────────────────────────────────────

                                                                                                                   
                                                 Dataset Overview                                                  
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ number of records               ┃ number of columns               ┃ percent complete records                    ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ 2                               │ 12                              │ 100.0%                                      │
└─────────────────────────────────┴─────────────────────────────────┴─────────────────────────────────────────────┘
                                                                                                                   
                                                                                                                   
                                              🌱 Seed-Dataset Columns                                              
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ column name                          ┃               data type ┃                           number unique values ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ diagnosis                            │                  string │                                     2 (100.0%) │
├──────────────────────────────────────┼─────────────────────────┼────────────────────────────────────────────────┤
│ patient_summary                      │                  string │                                     2 (100.0%) │
└──────────────────────────────────────┴─────────────────────────┴────────────────────────────────────────────────┘
                                                                                                                   
                                                                                                                   
                                                🎲 Sampler Columns                                                 
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ column name                   ┃       data type ┃             number unique values ┃               sampler type ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ patient_sampler               │            dict │                       2 (100.0%) │          person_from_faker │
├───────────────────────────────┼─────────────────┼──────────────────────────────────┼────────────────────────────┤
│ doctor_sampler                │            dict │                       2 (100.0%) │          person_from_faker │
├───────────────────────────────┼─────────────────┼──────────────────────────────────┼────────────────────────────┤
│ patient_id                    │          string │                       2 (100.0%) │                       uuid │
├───────────────────────────────┼─────────────────┼──────────────────────────────────┼────────────────────────────┤
│ symptom_onset_date            │          string │                       2 (100.0%) │                   datetime │
├───────────────────────────────┼─────────────────┼──────────────────────────────────┼────────────────────────────┤
│ date_of_visit                 │          string │                       2 (100.0%) │                  timedelta │
└───────────────────────────────┴─────────────────┴──────────────────────────────────┴────────────────────────────┘
                                                          

### 🆙 Scale up!

- Happy with your preview data?

- Use the `create` method to submit larger Data Designer generation jobs.


In [11]:
job_results = data_designer_client.create(config_builder, num_records=10)

[18:18:19] [INFO] 🎨 Creating Data Designer dataset


[18:18:19] [INFO] ✅ Validation passed


[18:18:19] [INFO] ⛓️ Sorting column configs into a Directed Acyclic Graph


[18:18:19] [INFO] 📂 Dataset path '/home/runner/work/DataDesigner/DataDesigner/docs/notebook_source/artifacts/dataset' already exists. Dataset from this session
		     will be saved to '/home/runner/work/DataDesigner/DataDesigner/docs/notebook_source/artifacts/dataset_12-12-2025_181819' instead.


[18:18:19] [INFO] 🩺 Running health checks for models...


...

[18:18:43] [INFO]   |-- 🧩 column: 'dob'


[18:18:43] [INFO]   |-- 🎲 column: 'symptom_onset_date'


[18:18:43] [INFO]   |-- 🎲 column: 'date_of_visit'


[18:18:43] [INFO]   |-- 🧩 column: 'physician'


[18:18:43] [INFO]   |-- 📝 column: 'physician_notes'


In [12]:
# Load the generated dataset as a pandas DataFrame.
dataset = job_results.load_dataset()

dataset.head()

,diagnosis,patient_summary,patient_sampler,doctor_sampler,patient_id,symptom_onset_date,date_of_visit,first_name,last_name,dob,physician,physician_notes
0,cervical spondylosis,I've been having a lot of pain in my neck and ...,"{'age': 83, 'bachelors_field': 'no_degree', 'b...","{'age': 68, 'bachelors_field': 'no_degree', 'b...",PT-915303CD,2024-10-04,2024-10-26,Tamara,Watkins,1942-09-08,Dr. Cameron,**Dr. Philip Cameron** **Primary Care Physic...
1,impetigo,I have a rash on my face that is getting worse...,"{'age': 80, 'bachelors_field': 'stem_related',...","{'age': 109, 'bachelors_field': 'no_degree', '...",PT-57565241,2024-10-14,2024-10-15,Jeffrey,Barrera,1945-04-30,Dr. Delgado,**Dr. Ricky Delgado** **Primary Care Physici...
2,urinary tract infection,I have been urinating blood. I sometimes feel ...,"{'age': 48, 'bachelors_field': 'no_degree', 'b...","{'age': 74, 'bachelors_field': 'no_degree', 'b...",PT-01BB27D4,2024-06-02,2024-06-10,Alexis,Johnston,1977-09-17,Dr. Lee,**Dr. Charles Lee** **Primary Care Physician...
3,arthritis,I have been having trouble with my muscles and...,"{'age': 81, 'bachelors_field': 'no_degree', 'b...","{'age': 99, 'bachelors_field': 'no_degree', 'b...",PT-3B5EBD50,2024-10-01,2024-10-13,Alison,Lambert,1944-02-07,Dr. Rocha,**Dr. William Rocha** **Primary Care Physici...
4,dengue,I have been feeling really sick. My body hurts...,"{'age': 110, 'bachelors_field': 'no_degree', '...","{'age': 96, 'bachelors_field': 'stem', 'birth_...",PT-FF2F9BE2,2024-01-07,2024-01-08,Michael,Adams,1915-01-10,Dr. Cunningham,**Dr. Robert Cunningham** **Primary Care Phy...


In [13]:
# Load the analysis results into memory.
analysis = job_results.load_analysis()

analysis.to_report()

──────────────────────────────────────── 🎨 Data Designer Dataset Profile ─────────────────────────────────────────

                                                                                                                   
                                                 Dataset Overview                                                  
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ number of records               ┃ number of columns               ┃ percent complete records                    ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ 10                              │ 12                              │ 100.0%                                      │
└─────────────────────────────────┴─────────────────────────────────┴─────────────────────────────────────────────┘
                                                                                                                   
                                                                                                                   
                                              🌱 Seed-Dataset Columns                                              
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ column name                          ┃               data type ┃                           number unique values ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ diagnosis                            │                  string │                                      7 (70.0%) │
├──────────────────────────────────────┼─────────────────────────┼────────────────────────────────────────────────┤
│ patient_summary                      │                  string │                                    10 (100.0%) │
└──────────────────────────────────────┴─────────────────────────┴────────────────────────────────────────────────┘
                                                                                                                   
                                                                                                                   
                                                🎲 Sampler Columns                                                 
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ column name                   ┃       data type ┃             number unique values ┃               sampler type ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ patient_sampler               │            dict │                      10 (100.0%) │          person_from_faker │
├───────────────────────────────┼─────────────────┼──────────────────────────────────┼────────────────────────────┤
│ doctor_sampler                │            dict │                      10 (100.0%) │          person_from_faker │
├───────────────────────────────┼─────────────────┼──────────────────────────────────┼────────────────────────────┤
│ patient_id                    │          string │                      10 (100.0%) │                       uuid │
├───────────────────────────────┼─────────────────┼──────────────────────────────────┼────────────────────────────┤
│ symptom_onset_date            │          string │                      10 (100.0%) │                   datetime │
├───────────────────────────────┼─────────────────┼──────────────────────────────────┼────────────────────────────┤
│ date_of_visit                 │          string │                      10 (100.0%) │                  timedelta │
└───────────────────────────────┴─────────────────┴──────────────────────────────────┴────────────────────────────┘
                                                          

## ⏭️ Next Steps

Use Data Designer to generate synthetic data for your specific use case!
